# Complete Notebook
## Module Loader

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
sns.set()
%matplotlib inline

In [ ]:
def sqft_to_m2(sqft):
    '''Converts sqft to m2
    Input: sqft (square feet value)
    Returns: converted m² value'''
    m2 = sqft / 10.764
    return m2

outlier_ids = []


## Analysis (General)
### Dataloader

In [ ]:
path = r"data\train.csv"
X_raw = pd.read_csv(path, index_col="Id")
X_ana = X_raw.copy()
X_test = pd.read_csv(path, index_col ="Id")
print(X_raw.columns)


### General analysis

**The Problem:**  
We want to predict houseprices based on several parameters of different segments.

Steps:  
A. Read the docs and get information about the dataset, its gathering and the environment of the dataset  
B. Analyse column by columns and make information about Type, Segment, Expected Importance  
C. Test the assumptions with box / scatter plots  
D. Write down consolidated information about importance  



A. Dataset information
Unfortunately no additional information were provided within the competition  
  
B. Analysis of every column
After revising every column and its describtion from the data_description.csv-file I have segmented and prioritized the different variabels.  
The most important ones form my perspective are:  
* LotArea
* Utilities
* Neighborhood
* BldgType
* OverallQual
* OverallCond
* TotalBsmtSF
* 1stFlrSF
* GrLivArea
* FullBath
* BedroomAbvGr
* TotRmsAbvGrd
* GarageArea

Next I will try to verify this assumptions with a correlation analysis

In [ ]:
#X_corr = X_ana.dropna(axis=1)
corrmat = X_ana.corr() #X_corr.corr()
fig, ax = plt.subplots(figsize=(12, 9))
ax1 = sns.heatmap(corrmat)
plt.show()
fig2, _ =plt.subplots(figsize=(12,9))
ax2 = sns.heatmap(corrmat.iloc[:, -1:].nlargest(100, corrmat.columns[-1]))
fig3, _ = plt.subplots(figsize=(25,18))

n_all = len(X_ana.columns)
n_specific = 16

cols = corrmat.nlargest(n_specific, "SalePrice")["SalePrice"].index 
cols_all = corrmat.nlargest(n_all, "SalePrice")["SalePrice"].index
#found missings in X_ana -> resulting in failure of corrcoef
#print(X_ana[cols].isna().any().sum())
#remove rows with missing values
X_ana_no_nan = X_ana.dropna(subset=cols_all, axis=0)

#use cleaned DF to proceed here
cm = np.corrcoef(X_ana_no_nan[cols_all].values, rowvar=False)
ax3 = sns.heatmap(cm, cbar=True,annot=True, xticklabels=cols_all.values, yticklabels=cols_all.values)

print(cols.values[1:16], cols.values[-16: -1])


The Top 15 most correlated columns according to this correlation analysis are  
'OverallQual'  
'GrLivArea'  
'GarageCars'  
'GarageArea'  
'TotalBsmtSF'  
'1stFlrSF'  
'FullBath'  
'TotRmsAbvGrd'  
'YearBuilt'  
'YearRemodAdd'  
'GarageYrBlt'  
'MasVnrArea'  
'Fireplaces'  
'BsmtFinSF1'  
'LotFrontage'   

The 15 least correlated columns according to this correlation analysis are  
'BsmtUnfSF'  
'BedroomAbvGr'  
'ScreenPorch'  
'PoolArea'  
'MoSold'  
'3SsnPorch'   
'BsmtFinSF2'  
'BsmtHalfBath'  
'MiscVal'  
'LowQualFinSF'  
'YrSold'    
'OverallCond'   
'MSSubClass'   
'EnclosedPorch'    

All others will be classified as Medium 

This insights results in an change of the potential importance of the features in the columns analysis, stated now in the Correlation column  
  
Possible siblings(high correlation) within high important variables are:  
* GarageArea and **GarageCars**:  
    The more cars fit in, the larger the garage has to be and vice versa  

* **TotalBsmtSF** and 1stFlrSF:  
    The first floor is mostly a similar size to the basement if existing  
      
* **YearBuilt**, YearRemodAdd and GarageYrBlt:  
    Most of the time the garage is build together with the house  
    YearRemodAdd is the same as the building year when not remodified since then  
    Best of them is YearBuilt 
  
  
* **GrLivArea** and TotRmsAbvGrd:  
    If the living area is bigger, potentially more rooms are in the house 
    Decision: GrLivArea  
  


In [ ]:
#removing the siblings from cols
cols_clean = list(set(cols) - set(["GarageArea", "1stFlrSF", "YearRemodAdd", "GarageYrBlt", "TotRmsAbvGrd"]))
cols_all_clean = list(set(cols) - set(["GarageArea", "1stFlrSF", "YearRemodAdd", "GarageYrBlt", "TotRmsAbvGrd"]))
print(cols_clean)

### SalePrice - Analysis of the dependent variable  
  
SalePrice will be the predicted variable. Let's take a look on it

In [ ]:
print(X_ana.SalePrice.describe())

looks like no 0 values (very good), positive skewness(not so good), some big ones at the end.  
Taking a picture of it including a normal dist to compare:

In [ ]:
fig, ax_dist = plt.subplots(figsize=(12,9), sharex=True)
mean = X_ana.SalePrice.mean()
median = X_ana.SalePrice.median()
sns.distplot(X_ana.SalePrice, fit=stats.norm, ax = ax_dist)
ax_dist.axvline(mean, color="red", linestyle="--")
ax_dist.axvline(median, color="green", linestyle = "-")
plt.legend({"Normal Dist":stats.norm, "Mean":mean, "Median":median})

Looks like a  
* deviation from normal   
* including high peak  
* with positive skewness  
* and outliers on the right (potentially above 700000).   

Measurements in numbers: 

In [ ]:
print("Kurtosis: %f" % X_ana.SalePrice.kurt())
print("Skewness: %f" % X_ana.SalePrice.skew())

Maybe we should transform it with a log function and look at it again


In [ ]:
#apply log
X_ana["SalePrice"] = np.log1p(X_ana["SalePrice"])

#inspect again
fig, ax_dist = plt.subplots(figsize=(12,9), sharex=True)
mean = X_ana.SalePrice.mean()
median = X_ana.SalePrice.median()
sns.distplot(X_ana.SalePrice, fit=stats.norm, ax = ax_dist)
ax_dist.axvline(mean, color="red", linestyle="--")
ax_dist.axvline(median, color="green", linestyle = "-")
plt.legend({"Normal Dist":stats.norm, "Mean":mean, "Median":median})

**Much better now!**

### Univariante analysis of all variables

In [ ]:
#reminder for the list of all variables
features = X_ana.copy()
all_cols = X_ana.columns.values
print(all_cols)

#define all numeric types
numeric_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

#print overview for all other types
for col in all_cols:
    print(X_ana[col].describe())
    print("# of unique values: %i" %X_ana[col].value_counts().count())
    print("# of NaNs: %i" %X_ana[[col]].isna().sum().values)
    if X_ana[col].dtype in numeric_types :
        sns.distplot(X_ana[col],hist= True, kde=False)
        print("Collerlation to SalePrice: %f" %corrmat.loc["SalePrice",col])
        
    else:
        sns.boxplot(x=col, y="SalePrice", data=X_ana, order=X_ana.groupby(col)["SalePrice"].mean().sort_values().index)
    plt.show()
    print("\n")    

    

In [ ]:
#### Finding the LotFrontage correlation

correlmat = X_raw.iloc[:, X_raw.columns != "SalePrice"].corr()
correlmat = correlmat.apply(lambda x: x.abs())
colsmax = correlmat["LotFrontage"].nlargest(150).index
#colsmin = correlmat["LotFrontage"].nsmallest(10).index
#cols = colsmax.append(colsmin)
cols = colsmax
cm2 = correlmat.loc[cols, cols]
sns.heatmap(cm2, xticklabels=cols, yticklabels=cols)
#specific_value = correlmat.loc["LotFrontage", "Neighborhood"]
print(cols)

stdevs_LotFrontage = {}
for col in X_raw.select_dtypes("object"):
    std_value = X_raw.groupby(col)["LotFrontage"].mean().std()
    stdevs_LotFrontage[col]= std_value
    #print("Variable: {} with std of {}".format(col, std_value))
#print(X_raw.groupby(["LotShape", "LotConfig","BldgType"])["LotFrontage"].median())


How can we describe the relation between the Top 15 relatives and SalePrice?

Using jointplots to take a deeper look at it:

In [ ]:
#for i, col in enumerate([c for c in cols_clean if c != "SalePrice"]):
#    fig, _ = plt.subplots(figsize=(12,9))
#    sns.scatterplot(x=col, y="SalePrice", data = X_ana)
#    plt.show()
sns.pairplot(data = X_ana[cols_clean[:3]], corner=True, kind="reg")


Now a detailed analysis for each of the remainind Top columns via visual inspection starting with **OverallQual**:

In [ ]:
#OverallQual
sns.jointplot(x="OverallQual", y="SalePrice", data = X_ana )

OverallQual seems to be exponentially related to Sales price with a wider spread at the upper end of the spectrum  
  
**Conclusion:** looks like a linear correlation to log(SalePrice)  
  
Next one will be 'GrLivArea'

In [ ]:
#GrLivArea
sns.jointplot(x="GrLivArea", y="SalePrice", data = X_ana, height=9, ratio=6 )

The relation could be a really steep linear with tendency to log relation based on the upper spectrum of X.  
Log-Transformation does the trick here.



In [ ]:
X_ana["GrLivArea" ] = np.log(X_ana["GrLivArea"])
sns.jointplot(x="GrLivArea", y="SalePrice", data = X_ana, height=9, ratio=6 )

No it looks quite nice, doesn't it?

Next one is GarageCars

In [ ]:
sns.jointplot(x="GarageCars", y="SalePrice", data=X_ana, kind="reg",height=9, ratio=6 )

This distributition looks very linear to me with a few outliers on the upper end (somekind of pattern...)

Next one: TotalBsmtSF

In [ ]:
sns.jointplot(x='TotalBsmtSF', y="SalePrice", data=X_ana, height = 9, ratio= 6)

In [ ]:
print(sqft_to_m2(6100))
TBSF_out = X_ana[X_ana["TotalBsmtSF"]>6000].index.values
print("ID with mega big basement: %i" % TBSF_out)

Aright: There must be a house (ID 1299) with a really, realy big basement :D Like 6100 sqft or 567 m² big.  
Beside that, the curve looks loggy to me.
On the other hand is a larger group of zeros present. This could be an indication for houses without basement.

Transformation:  
* additional column [HasBasement] with 0 or 1 if value of TotalBsmtSF = 0 or > 0
* change of values = 0 to 1
* np.log for values > 0


In [ ]:
X_ana["HasBasement"] = X_ana['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
X_ana["TotalBsmtSF"] = X_ana['TotalBsmtSF'].apply(lambda x: -999 if x == 0 else x)
X_ana["TotalBsmtSF"] = np.log(X_ana["TotalBsmtSF"])
sns.jointplot(x='TotalBsmtSF', y="SalePrice", data=X_ana, height = 9, ratio= 6)

Looks way better.

Next one: Year Build

In [ ]:
#plot pairplot
sns.catplot(kind="box", x="YearBuilt", y="SalePrice", data=X_ana, height=9)

Looks like a slightly exponentail growth over time but this cannot be verified due to lack of information about potential inflation considered in the prices

Next one: MasVnrArea

In [ ]:
#scatterplot of MasVnrArea against log(SalePrice)
sns.jointplot(x="MasVnrArea", y="SalePrice", data=X_ana, height=9, ratio=6)
n_zeros = X_ana["MasVnrArea"].value_counts().sort_index()[0]
print("Count of 0 in MasVnrArea: %i" % n_zeros)
print("Count of rest: %i" % (len(X_ana)-(n_zeros)))

There are a lot of houses without Masonry. The solution could be a colum has Masonry or not and dropping the original one. ->column simplification
On the other hand, the remaining 599 houses with data about masonry are in a medium correlation to Sales Price...  
  
Transformation:  
Do nothing about...

Next one:





In [ ]:
print(cols_clean)

### Further Analysis


Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate. XGBoost has a very useful function called as “cv” which performs cross-validation at each boosting iteration and thus returns the optimum number of trees required.
Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.
Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.
Lower the learning rate and decide the optimal parameters .


from sklearn.compose import Tar

#reg = XGBRegressor(early_stopping_counts=5, eval_stop=([X_valid, y_valid]) )
reg = ElasticNet()

X_train, X_valid, y_train, y_valid = train_test_split( X, y, train_size = 0.8, test_size=0.2, random_state=0)



#numeric_transformer = Pipeline(["imputer",SimpleImputer(strategy="median")])



#raising value error due to lack in synchronization caused by OneHotEncoder and OrdinalEncoder
preprocessing = ColumnTransformer(transformers=[("numeric", SimpleImputer(strategy = "median"), col_num) ,
                                            ("obj_below_10", Pipeline([("obj_imputer1", SimpleImputer(strategy = "most_frequent")),
                                                                        ("OHE"            , OneHotEncoder(handle_unknown='ignore'
                                                                                                            ))
                                                                        ]), col_obj_below_10) ,
                                            ("Neighborhood_", Pipeline([("obj_imputer2", SimpleImputer(strategy = "most_frequent")),
                                                                        ("label_encoder"  , OrdinalEncoder(categories=[list(X["Neighborhood"].unique())])
                                                                                                            )
                                                                        ]), ["Neighborhood"]),
                                            ("Exterior_2nd", Pipeline([("obj_imputer2", SimpleImputer(strategy = "most_frequent")),
                                                                        ("label_encoder"  , OrdinalEncoder(categories=[list(X["Exterior2nd"].unique())]))
                                                                        ]), ["Exterior2nd"])], remainder='drop')


my_pipe = Pipeline([("preprocessing",preprocessing),
                     ("pca", PCA(n_components = 42)),
                     ("reg",reg)])


param_grid = {  #"preprocessing__numeric__strategy":["median", "mean"],
                "pca__n_components": np.linspace(25, 65, 10, dtype="int"),
                #"reg__n_estimators": np.linspace(100, 1000, 20, dtype="int"),
                "reg__alpha":np.linspace(.01,1.5,50),
                "reg__l1_ratio": np.linspace(.2, .8, 10)
                }
cv = 5

search = GridSearchCV(my_pipe, param_grid=param_grid, scoring="neg_mean_absolute_error", verbose=5, n_jobs=-2, cv = cv)
search.fit(X_train,y_train)
print(abs(search.best_score_))
search_pred = search.predict(X_valid)
search_score = mean_absolute_error(y_valid, search_pred)
print(search.best_params_)

reg = ElasticNet(alpha=.253265, l1_ratio=0.66666666)
my_pipe = Pipeline([("preprocessing",preprocessing),
                     ("pca", PCA(n_components = 42)),
                     ("reg",reg)])
my_pipe.fit(X_train, y_train)
y_pred = my_pipe.predict(X_valid)
score = mean_absolute_error(y_valid, y_pred)
print(score)

rfr, lars, elnet, svr, ada, mlpr, xgbr = RandomForestRegressor(), LassoLars(), ElasticNet(), SVR(), AdaBoostRegressor(), MLPRegressor(), XGBRegressor()

model_instances = [rfr, lars, elnet, svr, ada, mlpr, xgbr]
rfr_list =["n_estimators"]
rfr_list.append(np.linspace(10, 200, 10, dtype="int"))
lars_list = ["alpha"]
lars_list.append(np.linspace(.01, .2, 10))
elnet = 

models = {}
grids = []
print(rfr_list)